# Class Exercises - Week 8

## Exercise 1 - Advertisement Optimization

When a user goes to a website, one of a set of $n$ ads, labeled $1,\dots,n$, is displayed. This is called an *impression*. We divide some time interval (say, one day) into $T$ periods, labeled $t = 1,\dots,T$. Let $N_{it} \ge 0$ denote the number of impressions in period $t$ for which we display ad $i$. In period $t$ there will be a total of $I_t >0$ impressions, so we must have $\sum_{i=1}^n N_{it} = I_t$, for $t=1,\dots,T$ (these numbers are known from past history). You can treat all these numbers as real (this is justified since they are typically very large).

The revenue for displaying ad $i$ in period $t$ is $R_{it} \ge 0$ per impression (this might come from click-through payments, for example). The total revenue is $\sum_{t=1}^T \sum_{i=1}^n R_{it}N_{it}$. To maximize revenue, we would simply display the ad with the highest revenue per impression, and no other, in each display period.

But our problem is more complicated. We also have in place a set of $m$ contracts that require us to display certain numbers of ads, or mixes of ads (say, associated with the products of one company), over certain periods, with a penalty for any shortfalls. Contract $j$ is characterized by a set of ads $\mathcal{A}_j \subseteq \{1,\dots,n\}$, a set of periods $\mathcal{T}_j \subseteq \{1,\dots,T\}$, a target number of impressions $q_j \ge 0$, and a shortfall penalty rate $p_j >0$.

The *shortfall* for contract $j$ is

$$
s_j = \left(q_j - \sum_{t\in \mathcal{T}_j} \sum_{i\in\mathcal{A}_j} N_{it} \right)_+,
$$

where $(u)_+$ means $\max\{u,0\}$. Our contracts require a total penalty payment equal to $\sum_{j=1}^m p_j s_j$. Our net profit is the total revenue minus the total penalty payment.

The data in this problem are $R\in \mathbb{R}^{n\times T}$, $I\in \mathbb{R}^T$ (here $I$ is the vector of impressions, not the identity matrix), and the contract data $\mathcal{A}_j$, $\mathcal{T}_j$, $q_j$, and $p_j$, for $j=1,\dots,m$. The data is generated by the snippet of code below. The data $\mathcal{A}_j$ and $\mathcal{T}_j$ for $j=1,\dots,m$ are given by matrices $A^{contr} \in\mathbb{R}^{n\times m}$ and $T^{contr} \in \mathbb{R}^{T\times m}$, with

$$
A_{ij}^{contr} = \left\{\begin{array}{ll}1 & i\in\mathcal{A}_j\\ 0 & \text{otherwise}, \end{array}\right. \qquad T_{tj}^{contr} = \left\{\begin{array}{ll}1 & t\in\mathcal{T}_j\\ 0 & \text{otherwise}, \end{array}\right.
$$



In [1]:
# Code to generate the data
import cvxpy as cp
import numpy as np

n=100 #number of ads
m=30 #number of contracts
T=60 #number of periods

np.random.seed(0)
I = 10*np.random.rand(T) #number of impressions in each period
R = np.random.rand(n,T) #revenue rate for each period and ad
q = T/n*50*np.random.rand(m) #contract target number of impressions
p = np.random.rand(m) #penalty rate for shortfall
Tcontr=(np.random.rand(T,m)>0.8) #one column per contract. 1's at the periods to be displayed
Acontr = np.zeros((n,m))
for i in range(n):
  contract=int(np.ceil(m*np.random.rand()))-1
  Acontr[i,contract]=1 #one column per contract. 1's at the ads to be displayed


# What is our variable: Nit matrix
N = cp.Variable((n,T))


#s = q - Tcontr@Acontr.T@N

q_real = np.zeros(m)

for i in range(m): # contracts
  for k in range(n): # ads
    #calc num impressions for adi 
    temp = 0
    if (Acontr[k,i] == True):
      temp += Tcontr[:,i] @ N[k,:].T
    q_real[i] = temp
  

(CVXPY) Oct 02 04:00:00 PM: Encountered unexpected exception importing solver SCS:
ImportError("dlopen(/Users/feyzjan/miniconda3/lib/python3.11/site-packages/_scs_direct.cpython-311-darwin.so, 0x0002): Library not loaded: @rpath/liblapack.3.dylib\n  Referenced from: <A08A2CF9-B9A1-393C-A32E-68987F02C61E> /Users/feyzjan/miniconda3/lib/python3.11/site-packages/_scs_direct.cpython-311-darwin.so\n  Reason: tried: '/Users/feyzjan/miniconda3/lib/python3.11/site-packages/../../liblapack.3.dylib' (no such file), '/Users/feyzjan/miniconda3/lib/python3.11/site-packages/../../liblapack.3.dylib' (no such file), '/Users/feyzjan/miniconda3/bin/../lib/liblapack.3.dylib' (no such file), '/Users/feyzjan/miniconda3/bin/../lib/liblapack.3.dylib' (no such file), '/usr/local/lib/liblapack.3.dylib' (no such file), '/usr/lib/liblapack.3.dylib' (no such file, not in dyld cache)")


ValueError: setting an array element with a sequence.

Use convex optimisation to find the display numbers $N_{ij}$ that maximise the net profit. In particular:
1.   What is the optimal net profit?
2.   What is the revenue associated with the optimal net profit?
3.   What is the total penalty payment associated with the optimal net profit?

Other's

## Solution 1

In [97]:
# What is our variable: Nit matrix
N = cp.Variable((n,T))


#s = q - Tcontr@Acontr.T@N

q_real = np.zeros(m)

for i in range(m): # contracts
  for k in range(n): # ads
    #calc num impressions for adi 
    temp = 0
    if (Acontr[k,i] == True):
      temp += Tcontr[:,i] @ N[k,:].T
    q_real[i] = temp
  
contract_penalty = cp.maximum((q - q_real),0)

revenue = 0

for i in range(n): # each ad
  for k in range(n): # each time
    revenue += R[i,k] * N[i,k]

obj = cp.Maximize(revenue- contract_penalty) 

constraints = [ cp.sum(N,axis=0) == I, # number of impressions per period is equal to sum of individual impressions of each ad per period              
]


#prob = cp.solve(obj,constraints)
               
    

AttributeError: ignored

In [99]:
q_real[0]

0.0

In [100]:
temp

0

In [84]:
Tcontr[:,0] @ N[0,:].T

7.642686482296174

others

### Solution Exercise 1

The optimal profit can be computed by solving the problem

$$
\begin{array}{ll}
\displaystyle \max_{N_{ij}} & \displaystyle\sum_{t=1}^T \sum_{i=1}^n R_{it}N_{it} - \sum_{j=1}^m p_j \left(q_j - \sum_{t\in \mathcal{T}_j} \sum_{i\in\mathcal{A}_j} N_{it} \right)_+ \\ 
\text{s.t. } & \sum_{i=1}^n N_{it} = I_t, \qquad t=1,\dots,T\\
& N\succeq 0.
\end{array}
$$

The constraints are linear. The shorfalls $s_j$ are convex functions of $N$, so the objective function is concave. The maximisation of a concave function is a convex problem. So we can solve this problem in CVX.

In [ ]:
N = cp.Variable((n,T)) #declare the variables N_it

In [ ]:
# We compute the total penalty
TotPen = 0

for j in range(m):
  tmp = p[j]*q[j] #maximum penalty for a fixed contract j
  for i in range(n):
    for t in range(T):
      if Acontr[i,j]*Tcontr[t,j] == 1: #if an ad in A_j is displayed when required
        tmp = tmp - p[j]*N[i,t] #we decrease the penalty
  TotPen = TotPen + cp.maximum(tmp,0) #total penalty, sum only if there is positive penalty 

Note that a more compact way of computing the vector of shortfalls is by using

$$
s = \left(q - \textbf{diag}\left(A^{contr \top}N T^{contr}\right) \right)_+
$$

In [ ]:
#TotPen = p@cp.maximum(q-cp.diag(Acontr.T@N@Tcontr),0)

In [ ]:
TotRev = cp.sum(cp.multiply(R,N)) # total revenue
NetProf = TotRev - TotPen # net profit

In [ ]:
# constraints are non-negativity of N_it and maximum number of impressions.
constraints = [N >= 0,cp.sum(N,0) == I]
# constraints = [N>=0, np.ones(n)@N==I]

prob = cp.Problem(cp.Maximize(NetProf),constraints)
prob.solve()

print("The optimal net profit is: ", prob.value)

The optimal net profit is:  259.0008355648624


In [ ]:
# We compute the optimal revenue using the found solution
OptRev = 0
for i in range(n):
  for t in range(T):
    OptRev = OptRev + R[i,t]*N.value[i,t]

print("The optimal revenue is: ", OptRev)

The optimal revenue is:  280.94172210073395


In [ ]:
# Or equivalently
print("The optimal revenue is: ", np.sum(np.multiply(R,N.value)))

The optimal revenue is:  280.94172210073447


In [ ]:
# We compute the optimal penalty using the found solution
OptPen = 0

for j in range(m):
  tmp = p[j]*q[j] 
  for i in range(n):
    for t in range(T):
      if Acontr[i,j]*Tcontr[t,j] == 1:
        tmp = tmp - p[j]*N.value[i,t]
  OptPen = OptPen + max(tmp,0)

print("The optimal penalty is: ", OptPen)

The optimal penalty is:  21.94088653587201


In [ ]:
# Or equivalently
print("The optimal penalty is: ", p@np.maximum(q-np.diag(Acontr.T@N.value@Tcontr),0))

The optimal penalty is:  21.940886535872014


## Exercise 2 - Advertisement Optimization Continued

This exercise is a continuation of the previous.

1.   What is the net profit if we were to display only the ad with the largest revenue per impression?
2.   What is the revenue if we were to display only the ad with the largest revenue per impression?
3.   What is the total penalty if we were to display only the ad with the largest revenue per impression?

### Solution Exercise 2

In [ ]:
# for each time t, we find the ad with the largest revenue
# and set the number of impressions for that ad to the maximum allowed in that period.
# All the other ads in that period get 0 impressions
NGreedy = np.zeros((n,T))
for t in range(T):
  NGreedy[np.argmax(R[:,t]),t] = I[t]

In [ ]:
# Same code as above to compute revenue, penalty and net profit
RevGrd = 0
for i in range(n):
  for t in range(T):
    RevGrd = RevGrd + R[i,t]*NGreedy[i,t]

print("The greedy revenue is: ", RevGrd)

The greedy revenue is:  306.36536218191895


In [ ]:
# Or equivalently
print("The greedy revenue is: ", np.sum(np.multiply(R,NGreedy)))

The greedy revenue is:  306.36536218191895


In [ ]:
# Same code as above to compute revenue, penalty and net profit
PenGrd = 0
for j in range(m):
  tmp = p[j]*q[j] 
  for i in range(n):
    for t in range(T):
      if Acontr[i,j]*Tcontr[t,j] == 1:
        tmp = tmp - p[j]*NGreedy[i,t]
  PenGrd = PenGrd + max(tmp,0)

print("The greedy penalty is: ", PenGrd)

The greedy penalty is:  159.48730250985878


In [ ]:
# Or equivalently
print("The greedy penalty is: ", p@np.maximum(q-np.diag(Acontr.T@NGreedy@Tcontr),0))

The greedy penalty is:  159.48730250985878


In [ ]:
print("The greedy net profit is: ", RevGrd-PenGrd)

The greedy net profit is:  146.87805967206017
